In [1]:
# load notebook details and inject API credentials (if available)
from edc import prepare; prepare("geodb-sharing")


***Notebook Title***  
GeoDB: Sharing Data

***Notebook Description***  
Euro Data Cube GeoDB: Sharing Data


***Notebook Dependencies***  
This notebook requires an active subscription to:
* EDC GeoDB

---------

*API credentials have automatically been injected for your active subscriptions.*

The following environment variables are now available:
* `SH_INSTANCE_ID`
* `SH_CLIENT_ID`
* `SH_CLIENT_SECRET`
* `SH_CLIENT_NAME`
* `GEODB_API_SERVER_URL`
* `GEODB_AUTH_CLIENT_ID`
* `GEODB_AUTH_AUD`
* `GEODB_AUTH_CLIENT_SECRET`
* `GEODB_API_SERVER_PORT`
* `GEODB_AUTH_DOMAIN`

-------------


## Sharing Data

In [2]:
from xcube_geodb.core.geodb import GeoDBClient

In [3]:
geodb = GeoDBClient()
geodb.whoami

'geodb_0c0a0ef4-8d0d-4e70-b002-033c7cbb41fd'

### Publish Dataset to the world

In [4]:
geodb.list_grants()

,table_name,grantee
0,geodb_0c0a0ef4-8d0d-4e70-b002-033c7cbb41fd_lan...,PUBLIC
1,geodb_0c0a0ef4-8d0d-4e70-b002-033c7cbb41fd_lan...,PUBLIC
2,geodb_0c0a0ef4-8d0d-4e70-b002-033c7cbb41fd_lan...,PUBLIC
3,geodb_0c0a0ef4-8d0d-4e70-b002-033c7cbb41fd_lan...,PUBLIC


In [5]:
geodb.publish_collection("land_use")

Access granted on land_use to public

In [6]:
geodb.list_grants()

,table_name,grantee
0,public_land_use,PUBLIC
1,public_land_use,PUBLIC
2,public_land_use,PUBLIC
3,public_land_use,PUBLIC


In [7]:
geodb.get_collection('land_use', database='public')

,id,created_at,modified_at,geometry,raba_pid,raba_id,d_od
0,1,2020-04-23T14:25:57.670808+00:00,None,"POLYGON ((453952.629 91124.177, 453952.696 911...",4770326,1410,2019-03-26
1,9828,2020-05-18T07:57:40.617849+00:00,None,"POLYGON ((453952.629 91124.177, 453952.696 911...",4770326,1410,2019-03-26
2,19655,2020-05-18T08:11:17.715062+00:00,None,"POLYGON ((453952.629 91124.177, 453952.696 911...",4770326,1410,2019-03-26
3,4,2020-04-23T14:25:57.670808+00:00,None,"POLYGON ((455929.405 97963.785, 455933.284 979...",2305596,1100,2019-02-25
4,5,2020-04-23T14:25:57.670808+00:00,None,"POLYGON ((461561.512 96119.256, 461632.114 960...",2310160,1100,2019-03-11
...,...,...,...,...,...,...,...
28324,29419,2020-05-18T08:11:17.715062+00:00,2020-05-18T08:11:22.848066+00:00,"POLYGON ((458304.555 100146.103, 458306.726 10...",6268058,1300,2000-01-01
28325,29420,2020-05-18T08:11:17.715062+00:00,2020-05-18T08:11:22.848066+00:00,"POLYGON ((458247.583 99746.065, 458245.202 997...",6268059,1300,2000-01-01
28326,29421,2020-05-18T08:11:17.715062+00:00,2020-05-18T08:11:22.848066+00:00,"POLYGON ((461932.283 90520.703, 461931.619 905...",6263561,1300,2000-01-01
28327,29452,2020-05-18T08:11:17.715062+00:00,2020-05-18T08:11:22.848066+00:00,"POLYGON ((457231.170 90902.256, 457234.861 909...",6264862,1300,2000-01-01


### Revoke access

In [8]:
geodb.list_grants()

,table_name,grantee
0,public_land_use,PUBLIC
1,public_land_use,PUBLIC
2,public_land_use,PUBLIC
3,public_land_use,PUBLIC


In [9]:
geodb.unpublish_collection("land_use")

Access revoked for land_use from public

In [10]:
geodb.list_grants()

,table_name,grantee
0,geodb_0c0a0ef4-8d0d-4e70-b002-033c7cbb41fd_lan...,PUBLIC
1,geodb_0c0a0ef4-8d0d-4e70-b002-033c7cbb41fd_lan...,PUBLIC
2,geodb_0c0a0ef4-8d0d-4e70-b002-033c7cbb41fd_lan...,PUBLIC
